In [ ]:
from __future__ import unicode_literals

import sys, os
BIN = os.path.expanduser("../../")
sys.path.append(BIN)

import numpy as np
from scipy.constants import m_p, c, e, pi
import matplotlib.pyplot as plt
%matplotlib inline

import copy
import itertools

from test_tools import generate_objects, BunchTracker, track, compare_traces, compare_projections, Machine

from PyHEADTAIL_feedback.feedback import OneboxFeedback, Kicker, PickUp
from PyHEADTAIL_feedback.processors import ChargeWeighter, Averager, Bypass, Lowpass, Sinc, NoiseGenerator
from PyHEADTAIL_feedback.digital_processors import HilbertPhaseShiftRegister, ADC, DAC, FIR_Filter
from PyHEADTAIL.particles.slicing import UniformBinSlicer

np.random.seed(0)

In [ ]:
""" 
    In this example, a model for more realistic feedback system is presented. Note that it does not try to 
    emulate any system from the real life, but parts of the system required by more realistic simulations are
    presented. The simulations are very sensitive to details of the feedback model and it must be remembered
    that this is a toy model!
    
    The example is based on directly to one pickup and one kicker example in the file 
    002_separated_pickup_and_kicker.ipynb. This example focuses on the details of the signal processors, and 
    more details about other parts of the model can be found from there.
"""

In [ ]:
# basic parameters of the simulation
n_macroparticles = 10000
n_slices = 200
n_segments = 5
n_sigma_z = 3 

n_turns = 5000

bunch_ref, slicer_ref,trans_map, long_map = generate_objects(n_macroparticles,n_segments, n_slices,n_sigma_z)
bunch_unkicked = copy.deepcopy(bunch_ref)


In [ ]:
# This creates an artificially kicked bunch, which will be damped with different feedback systems
slicer_for_kicks = UniformBinSlicer(n_slices=5, n_sigma_z=3)
slice_set = bunch_ref.get_slices(slicer_for_kicks, statistics=True)
p_idx = slice_set.particles_within_cuts
s_idx = slice_set.slice_index_of_particle.take(p_idx)

# random kicks
kick_x = 0.003*(-1.0+2*np.random.rand(n_slices))
kick_y = 0.003*(-1.0+2*np.random.rand(n_slices))

for p_id, s_id in itertools.izip(p_idx,s_idx):
    bunch_ref.x[p_id] += kick_x[s_id]
    bunch_ref.y[p_id] += kick_y[s_id]
    
bunch_init = copy.deepcopy(bunch_ref)
tracker_ref = BunchTracker(bunch_init)
maps_ref = [i for i in trans_map] + [long_map]
track(n_turns, bunch_init,maps_ref,tracker_ref)

# The machine parameters have been determined in Machine() object in file test_tools.py
machine = Machine()

In [ ]:
# In the used code, the parameter 'gain' is determined as a fraction of the input signal if the used signal 
# processors would bypass the signalperfectly

# There are two ways to set a gain value. If only one value is given, same value will be used both
# in horizontal and vertical plane. If two values are given, separed values (x,y) are used for horizontal and
# vertical planes

feedback_gain = 0.033
# feedback_gain = (0.01,0.01)


# Parameters for the feedback system
pickup_bandwidth = 10e9 # [Hz]
sampling_rate = 4e9 # [Hz]
n_FIT_taps = 5
FIR_bandwidth = 1e9 # [Hz] 
kicker_bandwidth = 1e9 # [Hz]

noise_level = 1.
noise_level_x = noise_level * bunch_unkicked.sigma_x()
noise_level_y = noise_level * bunch_unkicked.sigma_y()


delay = 1 
n_values = 7

In [ ]:
# A reference feedback system for examples presented in this file is the ideal bunch feedback presented in 
# file 001_ideal_feedbacks.ipynb:

bunch_OneBox_bunch = copy.deepcopy(bunch_ref)
tracker_OneBox_bunch = BunchTracker(bunch_OneBox_bunch)
slicer_OneBox_bunch = copy.deepcopy(slicer_ref)

processors_bunch_x = [
    ChargeWeighter(normalization = 'average_weight'),
    Averager()
]
processors_bunch_y = [
    ChargeWeighter(normalization = 'average_weight'),
    Averager()
]

feedback_map = OneboxFeedback(feedback_gain,slicer_OneBox_bunch,processors_bunch_x,processors_bunch_y)
total_map_OneBox_bunch = [i for i in trans_map] + [long_map] + [feedback_map]

track(n_turns, bunch_OneBox_bunch,total_map_OneBox_bunch,tracker_OneBox_bunch)

# That feedback system is extended to consist of separated pickup and kicker in this file. If you don't
# understand details of the code above, please study examples in 001_ideal_feedbacks.ipynb

In [ ]:
bunch_separated_example = copy.deepcopy(bunch_ref)
tracker_separated_example = BunchTracker(bunch_separated_example)
slicer_separated_example = copy.deepcopy(slicer_ref)


processors_separated_pickup_x = [
    # Pickup plates:
    # If signal directly from the pickup plates is used, it is proportional to charge weighter positions 
    # of the slices. There are finite bandwidth for the pickup, and the simplest model for it is RC-lowpass
    # filter. Bandwidth of the pickup is typically significantly higher than bandwidths of the other parts 
    # of the system. 
    #
    # Every part of the system induces noise to the signal. In this example it is adde after the picup, but
    # it can be adde after every processor. Note that filters after the noise generator attenuates the noise
    #
    # In some systems charge weighting is removed by calculating ratio of sum and difference signals from two
    # picup plates (delta-sigma). In these cases a special signal processor is required, which can be found 
    # from https://gitlab.cern.ch/jakomppu/PyHEADTAIL_feedback.git
    ChargeWeighter(), 
    Lowpass(pickup_bandwidth),
    NoiseGenerator(noise_level_x),
    
    # Digital signal processing:
    # In many systems, the signal is mainly processed by using digital signal processing. The minimal model
    # for this is to use a FIR lowpass filter inside the ADC and DAC. The finite simpling rate and accuracy of
    # the dac can be modelled by giving those parameters to the ADC.
    ADC(sampling_rate), # Digital signal processing
    FIR_Filter(n_FIT_taps,FIR_bandwidth,sampling_rate),# Digital signal processing
    DAC(sampling_rate),# Digital signal processing
    
    # Actual betatron phase correction is typically done digitally between ADC and DAC. Because this model is
    # linear, this can be done also after the DAC.
    HilbertPhaseShiftRegister(n_values, machine.Q_x, delay)
]
processors_separated_pickup_y = [
    ChargeWeighter(),
    Lowpass(pickup_bandwidth),
    NoiseGenerator(noise_level_y),
    ADC(sampling_rate),
    FIR_Filter(n_FIT_taps,FIR_bandwidth,sampling_rate),
    DAC(sampling_rate),
    HilbertPhaseShiftRegister(n_values, machine.Q_y, delay)
]

pickup_position_x = 1.*2.*pi/float(n_segments)*machine.Q_x
pickup_position_y = 1.*2.*pi/float(n_segments)*machine.Q_y

pickup_map = PickUp(slicer_separated_example,processors_separated_pickup_x,processors_separated_pickup_y, 
       pickup_position_x, pickup_position_y)



# The model for kicker depends on the power amplifier and the kicker. Frequency response of the power
# amplifier might (partially) phase linearized in digital signal processing, or frequency response of the 
# amplifier might decay shortly after the cutoff frequency. Thus, any of the lowpass filters might be
# a good model, but Sinc filter might be the safest option.
processors_separated_kicker_x = [Sinc(kicker_bandwidth)]
processors_separated_kicker_y = [Sinc(kicker_bandwidth)]


# Rememeber to change the index, if the number of processors is changed in the processors list of the pickup
registers_x = [processors_separated_pickup_x[6]]
registers_y = [processors_separated_pickup_y[6]]

kicker_position_x = 2.*2.*pi/float(n_segments)*machine.Q_x
kicker_position_y = 2.*2.*pi/float(n_segments)*machine.Q_y

xp_per_x = bunch_unkicked.sigma_xp()/bunch_unkicked.sigma_x()
yp_per_y = bunch_unkicked.sigma_yp()/bunch_unkicked.sigma_y()

kicker_map = Kicker(feedback_gain, slicer_separated_example,
                    processors_separated_kicker_x, processors_separated_kicker_y,
                    kicker_position_x, kicker_position_y,
                    registers_x, registers_y, xp_per_x, yp_per_y)

total_map_separated_example = [trans_map[0]] + [pickup_map] + [trans_map[1]] + [kicker_map]
for element in trans_map[2:]:
    total_map_separated_example += [element]
total_map_separated_example += [long_map]
    
    
track(n_turns, bunch_separated_example,total_map_separated_example,tracker_separated_example)

In [ ]:
# If details of the feedback system and/or sensitivity of the model for those details is not known, it might
# be beter to use much simpler and stabler model for the feedback system. One options is Sinc filter, but it
# is good to remember that Sinc filter is not ideal as presented in the previous example 
# (004_analog_signal_processors.ipynb).

bunch_simplest_model = copy.deepcopy(bunch_ref)
tracker_simplest_model = BunchTracker(bunch_simplest_model)
slicer_simplest_model = copy.deepcopy(slicer_ref)

processors_simplest_model_x = [
    ChargeWeighter(),
    Sinc(kicker_bandwidth)
]
processors_simplest_model_y = [
    ChargeWeighter(),
    Sinc(kicker_bandwidth)
]

feedback_map = OneboxFeedback(feedback_gain,slicer_simplest_model,
                              processors_simplest_model_x,processors_simplest_model_y)
total_map_simplest_model = [i for i in trans_map] + [long_map] + [feedback_map]

track(n_turns, bunch_simplest_model,total_map_simplest_model,tracker_simplest_model)

In [ ]:
compare_traces([tracker_OneBox_bunch,tracker_separated_example,tracker_simplest_model],
               ['Ideal', 'Delayed', 'Separated'])
compare_projections([ bunch_OneBox_bunch, bunch_separated_example,  bunch_simplest_model], 
                    ['Ideal bunch', 'Complex model', 'Simplest model'])